# Задание 1

Опишите задачу с точки зрения NLP. Что это за задача, какие классические методы существуют для ее решения? Как ее можно решать через LLM? Как обычно оценивают качество моделей в этой задаче?


Это задача распознавания именных сущностей(Named entity recognition NER), задача nlp, где мы пытаемся в тексте понять, где находятся имена/номера домов/названия организаций/ и т.д.

Классические методы решения NER включают:

* Подходы, основанные на правилах: с помощью регулярок/шаблонов/лингвистических признаков.
* На основе машинного обучения: Модели, например BERT, обучаются на размеченных данных, чтобы после обучения они сами могли распознавать именованные сущности.
* Гибридные подходы: Могут сначала использоваться подходы на основе правил, а после на основе ml для уточнения.

Для оценки качества можно использовать метрики, как precision, recall, f-мера


# Задание 2
Реализуйте чтение датасета в pandas DataFrame с обязательными колонками "document_id", "document_text", "entity", "gold_answer". 

In [ ]:
from corus import load_bsnlp # я сначала сам пытался прочитать файл, но нашел замечательную либу, которая за меня это сделает)
interest_entity = ['PER', 'ORG', 'LOC', 'EVT', 'PRO']
path = 'data/' # путь к датасетам
doc_id = []
doc_text = []
entity = []
gold_answer = []
doc_len = []
records = load_bsnlp(path)
for record in records:
    for ner in record.substrings:
        if ner.id.split('-')[0] in interest_entity:
            doc_id.append(record.id)
            doc_text.append(ner.text)
            entity.append(ner.type)
            gold_answer.append(ner.id)
            doc_len.append(len(record.text))
        

In [ ]:
import pandas as pd

data = pd.DataFrame({
    'document_id': doc_id,
    'document_text': doc_text,
    'entity': entity,
    'gold_answer': gold_answer,
    'document_lenght': doc_len
})


In [ ]:
data

# Задание 3
Напишите функцию, которая принимает на вход строку??? датафрейма и выдает текст входного сообщения для LLM.

In [ ]:
def for_llm(document_text):
    return f"""You are a smart and intelligent Named Entity Recognition (NER) system. You need recognize PER, ORG, LOC, EVT, PRO.

<Q>Хадимом Хусейном Ризви
<A>
entity  gold_answer
PER     PER-Khadim-Hussain-Rizvi
<Q>Facebook
<A>
entity  gold_answer
PRO     PRO-Facebook
<Q>Heute
<A>
entity  gold_answer
ORG     ORG-Heute-News
<Q>{document_text}"""


In [ ]:
llm_text = data['document_text'].apply(for_llm)
for i in llm_text:
    print()
    print(i)
    print()

# Задание 4

Получите ответы GigaChat для всех документов. Документов всего 9, поэтому сделать это можно вручную, пользуясь веб-интерфейсом GigaChat или ботом в ВК или Телеграме. Не очищайте историю сообщений, чтобы потом продемонстрировать подлинность ответов на онлайн-собеседовании.
Внесите ответы GigaChat в датафрейм, сохраните его.


In [ ]:
# s = """Борис Джонсон	Борис Джонсон	PER	PER-Boris-Johnson
# Британии	Великобритания	LOC	GPE-Great-Britain
# Дэвида Дэвиса	Дэвид Дэвис	PER	PER-David-Davis
# ЕС	ЕС	ORG	ORG-European-Union
# Мэй	Мэй	PER	PER-Theresa-May
# Терезы Мэй	Тереза Мэй	PER	PER-Theresa-May
# британской прессы	британская пресса	ORG	ORG-British-Press
# выхода из ЕС	выход из ЕС	EVT	EVT-Brexit
# выхода из Евросоюза	выход из Евросоюза	EVT	EVT-Brexit"""
# needed = [line.split('\t')[-1] for line in s.split('\n')]
# needed

# Задание 5

In [ ]:
data['gold_answer'] = data['gold_answer'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
data['giga_pred'] = data['giga_pred'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [ ]:
def score_fn(gold: str, pred: str) -> float:
    gold_entities = [entity for entity in gold.split()]
    pred_entities = [entity for entity in pred.split()]
    n = 1
    precision = sum(1 for pred in pred_entities if pred in gold_entities) / len(pred_entities)
    recall = sum(1 for pred in pred_entities if pred in gold_entities) / len(gold_entities)
    f_score = ((1 + n**2) * precision * recall)/((n**2 * precision) + recall)
    return f_score

# Задание 6

In [ ]:
scores = data.apply(lambda x: score_fn(x['gold_answer'], x['giga_pred']), axis=1)
data['scores'] = scores

In [ ]:
import matplotlib.pyplot as plt
# Calculate scores for each row

# Group by entity and calculate mean scores
entity_scores_mean = scores.groupby(data['gold_answer']).mean()

# Group by document and calculate mean scores
document_scores_mean = scores.groupby(data.index).mean()

# Plot the results
plt.bar(entity_scores_mean.index, entity_scores_mean.values)
plt.xlabel('Entity')
plt.ylabel('Mean Score')
plt.title('Score by Entity')

In [ ]:
plt.bar(document_scores_mean.index, document_scores_mean.values)
plt.xlabel('Document')
plt.ylabel('Mean score')
plt.title('Score by document')

plt.tight_layout()

# Задание 7

In [ ]:
import matplotlib.pyplot as plt

# Calculate the length of each document
doc_lengths = data['document_text'].apply(len)

# Plot the scores against doc lengths
plt.scatter(doc_lengths, scores)
plt.xlabel('Document Length')
plt.ylabel('Score')
plt.show()

# Задание 8

In [ ]:
# Calculate the number of correct and incorrect answers for each document
correct_answers = data['gold_answer'] == data['giga_pred']
incorrect_answers = ~correct_answers

# Visualize the results
plt.bar(data['document_id'], correct_answers.sum(), label='Correct')
plt.bar(data['document_id'], incorrect_answers.sum(), label='Incorrect')
plt.xlabel('Document ID')
plt.ylabel('Number of Answers')
plt.legend()
plt.show()

# Задание 9
